In [1]:
%load_ext dotenv
%reload_ext dotenv
%dotenv

In [2]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

import folium
from folium.plugins import HeatMap
from shapely.geometry import MultiPoint

In [3]:
import os
import sys

from os.path import dirname
UTILS_PATH=os.environ.get('REPO') + "/notebooks/commons"
sys.path.append(dirname(UTILS_PATH))

from commons import check_args_variables, gen_paths_variables, coordinates_bounds

In [4]:
if __name__ == "__main__":
    source = "OSM"
    city = "manizales"
    
    base_path = os.environ.get('REPO')
    
    source_path = base_path + gen_paths_variables(source, city)

In [6]:
MSG = """
Hay {} publicaciones de {} en total
"""

external_variables = pd.read_parquet(source_path)
print(MSG.format(external_variables.shape[0], city))
external_variables.head()


Hay 693 publicaciones de manizales en total



,lat,lon,name,amenity
0,5.0644216,-75.4951894,CAI La Leonora,police
1,5.0646280,-75.4974278,101 Perros,fast_food
2,5.0613149,-75.4904344,Universidad de Caldas - Sede Palogrande,university
3,5.0716575,-75.5171473,None,marketplace
4,5.0668861,-75.5042989,Universidad Autónoma de Manizales,university


In [7]:
from sklearn.cluster import DBSCAN

In [8]:
coords = external_variables[['lat','lon']].copy()
coords.loc[:, "lat"] = coords["lat"].apply(float)
coords.loc[:, "lon"] = coords["lon"].apply(float)
coords = coords.values

kms_per_radian = 6371.0088
epsilon = 0.1 / kms_per_radian

min_samples = 20

In [9]:
db = DBSCAN(eps=epsilon, min_samples=min_samples, algorithm='ball_tree', metric='haversine').fit(np.radians(coords))

In [10]:
centroids = []
clusters_size = len(db.labels_)

for n in range(clusters_size):
    cluster = external_variables[['lat','lon']][db.labels_ == n].copy()
    if len(cluster) > 0:
        cluster.loc[:, "cluster_id"] = n
        
        multi_point = MultiPoint(cluster.values)
        cluster.loc[:, "cluster_latitude"] = multi_point.centroid.x
        cluster.loc[:, "cluster_longitude"] = multi_point.centroid.y
        
    centroids.append(cluster)
        
clusters_df = pd.concat(centroids)
clusters_df = clusters_df.reset_index(drop=True)
clusters_df.head()

,lat,lon,cluster_id,cluster_latitude,cluster_longitude
0,5.0568617,-75.4864150,0.0,5.056436,-75.486162
1,5.0562421,-75.4853083,0.0,5.056436,-75.486162
2,5.0564718,-75.4854604,0.0,5.056436,-75.486162
3,5.0564944,-75.4854278,0.0,5.056436,-75.486162
4,5.0557649,-75.4871391,0.0,5.056436,-75.486162


In [11]:
clusters_coordinates = clusters_df[["cluster_id", "lat", "lon"]].groupby(["lat", "lon"]).count()

coordinates = list(zip(*clusters_coordinates.index))
clusters_coordinates.loc[:, "lat"] = coordinates[0]
clusters_coordinates.loc[:, "lon"] = coordinates[1]

clusters_coordinates.loc[:, "count"] = clusters_coordinates["cluster_id"].apply(lambda e: float(e*100))

clusters_coordinates = clusters_coordinates.reset_index(drop=True)

clusters_coordinates.head()

,cluster_id,lat,lon,count
0,1,5.0552121,-75.4854908,100.0
1,1,5.0553625,-75.4851470,100.0
2,1,5.0555672,-75.4855483,100.0
3,1,5.0555889,-75.4866771,100.0
4,1,5.0557555,-75.4870577,100.0


In [12]:
folium_hmap = folium.Map(location=[
    np.mean([coordinates_bounds[city]["lat"]["lower"], coordinates_bounds[city]["lat"]["upper"]]),
    np.mean([coordinates_bounds[city]["lon"]["lower"], coordinates_bounds[city]["lon"]["upper"]])
], zoom_start=13, tiles="OpenStreetMap")

max_amount = clusters_coordinates['count'].max()

hm_wide = HeatMap( 
    clusters_coordinates[["lat", "lon", "count"]],
    min_opacity=0.2,
    max_val=max_amount,
    radius=8, blur=6, 
    max_zoom=15, 
    gradient={.1:'yellow', .5: 'orange',  1: 'red'}
)

folium_hmap.add_child(hm_wide)